In [2]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
import json
import numpy as np
from sklearn.model_selection import KFold, train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error
import matplotlib.pyplot as plt
from scipy.stats import entropy

def load_puzzle_data(file_path):
    !cd buggle-training-data && git pull
    with open('buggle-training-data/training_data.json', 'r') as f:
      puzzle_data = json.load(f)
      return puzzle_data

def extract_features(puzzle_data):
    matrix_features = []
    scalar_features = []
    for puzzle in puzzle_data:
        features = puzzle['features']
        try:
            matrix = features['matrix']
            if any(len(row) != 5 for row in matrix) or len(matrix) != 5:
                print("Error: Matrix is not 5x5")
                continue

            # Convert matrix to ASCII values and retain as 2D
            ascii_matrix = np.array([[ord(char) for char in row] for row in matrix])
            matrix_features.append(ascii_matrix)

            # Extract scalar features
            syllable_score = features['syllableScore']
            scalar = np.array([syllable_score])
            scalar_features.append(scalar)

        except KeyError as e:
            print(f"Missing key in puzzle: {e}")
            continue
        except Exception as e:
            print(f"Error processing puzzle: {e}")
            continue

    return np.array(matrix_features), np.array(scalar_features)

def preprocess_data(matrix_features, scalar_features, outcomes):
    # Scalar features are expanded to match matrix dimensions and concatenated
    scalar_features_expanded = scalar_features[:, np.newaxis, np.newaxis, :]
    scalar_features_expanded = np.repeat(scalar_features_expanded, 5, axis=1)
    scalar_features_expanded = np.repeat(scalar_features_expanded, 5, axis=2)

    if matrix_features.ndim == 3:
        matrix_features = matrix_features[..., np.newaxis]  # Add a channel dimension if not present

    # Concatenate along the last dimension
    X = np.concatenate([matrix_features, scalar_features_expanded], axis=3)

    if X.shape[-1] != 2:
      raise ValueError(f"Expected 5 channels, but got {X.shape[-1]} channels.")

    X_train, X_test, y_train, y_test = train_test_split(X, outcomes, test_size=0.2, random_state=42)

    return X_train, X_test, y_train, y_test

def build_model():
    model = Sequential([
        Input(shape=(5, 5, 5)),  # Adjust this to match the input shape
        Conv2D(32, (3, 3), activation='relu', padding='same'),
        MaxPooling2D((2, 2)),
        Dropout(0.25),
        Conv2D(64, (3, 3), activation='relu', padding='same'),
        MaxPooling2D((2, 2)),
        Dropout(0.25),
        Flatten(),
        Dense(128, activation='relu'),
        Dropout(0.5),
        Dense(1)  # Assuming a regression task for predicting 'totalWords'
    ])
    model.compile(optimizer='adamw', loss='mean_squared_error', metrics=['mean_absolute_error'])
    model.summary()
    return model

# Main function to run the script
def main():
    # Load data
    puzzle_data = load_puzzle_data('buggle-training-data/training_data.json')
    print(f"Got {len(puzzle_data)} puzzles.")
    matrix_features, scalar_features = extract_features(puzzle_data)
    print(f"Loaded {len(matrix_features)} matrix features and {len(scalar_features)} scalar features.")
    outcomes = np.array([puzzle['outcomes']['totalWords'] for puzzle in puzzle_data])
    print(f"Loaded {len(outcomes)} outcomes.")

    # Preprocess data
    X_train, X_test, y_train, y_test = preprocess_data(matrix_features, scalar_features, outcomes)
    print(f"Preprocessed {len(X_train)} training examples and {len(X_test)} test examples.")

    model = build_model()

    # Set up early stopping
    early_stopping_monitor = EarlyStopping(
        monitor='val_mean_absolute_error',
        patience=10,  # Number of epochs with no improvement after which training will be stopped
        restore_best_weights=True  # Restores model weights from the epoch with the best value of the monitored quantity
    )

    # Train the model with the early stopping callback
    history = model.fit(
        X_train, y_train,
        epochs=50,
        batch_size=16,
        validation_split=0.2,
        verbose=1,
        callbacks=[early_stopping_monitor]
    )

    print("Finished training.")

    # Evaluate the model
    test_results = model.evaluate(X_test, y_test, verbose=1)
    print(f"Test Loss: {test_results[0]}, Test MAE: {test_results[1]}")


    # Plot training history
    plt.figure(figsize=(8, 4))
    plt.plot(history.history['mean_absolute_error'], label='MAE (training data)')
    plt.plot(history.history['val_mean_absolute_error'], label='MAE (validation data)')
    plt.title('MAE for Puzzle Prediction')
    plt.ylabel('MAE value')
    plt.xlabel('No. epoch')
    plt.legend(loc="upper right")
    plt.show()

# Run the script
if __name__ == '__main__':
    main()

ModuleNotFoundError: No module named 'tensorflow'